# Prep OPERA RTC CalVal data: stage 1, part 1

**Alex Lewandowski; Alaska Satellite Facility, University of Alaska Fairbanks**

## Performs initial data preparation for OPERA RTC calibration and validation

**Notebook Requires**
- dual-pol backscatter geotiffs
- local incidence angle map
- ellipsoidal incidence angle map
- layover-shadow mask
- dem
- scene shape files
- Copernicus land cover classification map (multiple tiles may be included for complete AOI coverage)

**Actions**
- merges land cover classification maps
- subsets all geotiffs to scene
- masks water bodies in land cover map (sets values 80 & 200 -> 0)

In [ ]:
from ipyfilechooser import FileChooser
import numpy as np
from pathlib import Path
import shutil

import numpy.ma as ma
from osgeo import gdal

## Select the directory holding your RTC product directory and Copernicus land cover Discrete-Classification maps

Note: dems, local incidence angle maps, ellipsoidal incidence maps, layover-shadow maps, dual-pol backscatter geotiffs, scene shape files, and Copernicus land data are needed

```
data_directory   
│
└───product_1_directory
│   │   *_VH.tif
│   │   *_VV.tif
│   │   *lc_inc_map.tif
│   │   *el_inc_map.tif
│   │   *ls_map.tif
│   │   *_shape.dbf
│   │   *_shape.prj
│   │   *_shape.shp
│   │   *_shape.shx
│   │   *_dem.tif
│   ...
│   
└───*_Discete-Classification-map-EPSG-*.tif
│ 
└───*_Discete-Classification-map-EPSG-*.tif
│ 
...
```

In [ ]:
print("Select the directory holding your data with the file structure shown above")
fc = FileChooser(Path.cwd())
display(fc)

In [ ]:
data_dir = Path(fc.selected_path)
product_name = list(data_dir.glob("*/*_VH.tif"))[0].parent
output_dir = data_dir.parent/f"{product_name}_prepped"
if not output_dir.is_dir():
    output_dir.mkdir()

## Merge Copernicus Global Land Cover Tiles if necessary

In [ ]:
land_cover = list(data_dir.glob('*_Discrete-Classification*.tif'))
land_cover

In [ ]:
if len(land_cover) > 1:
    merge_str = ''
    epsg = land_cover[0].stem.split('EPSG-')[1]
    for tiff in land_cover:
        merge_str = f"{merge_str} {str(tiff)}"
    gdal_cmd = f"gdal_merge.py -o {data_dir}/Discrete-Classification-map_EPSG-{epsg}.tif {merge_str}"
    !$gdal_cmd
    land_cover = list(data_dir.glob('Discrete-Classification-map_EPSG-*.tif'))[0]
else:
    land_cover = land_cover[0]

## Subset Copernicus Global Land Cover Data and Local Incidence Angle Map using HyP3 product shape file

In [ ]:
shp = list(data_dir.glob('*/*_shape.shp'))
local_inc = list(data_dir.glob('*/*_lc_inc_map.tif'))[0]
elipse_inc = list(data_dir.glob('*/*_ell_inc_map.tif'))[0]
ls_map = list(data_dir.glob('*/*_ls_map.tif'))[0]
vh = list(data_dir.glob("*/*_VH.tif"))[0]
vv = list(data_dir.glob("*/*_VV.tif"))[0]
dem = list(data_dir.glob("*/*_dem.tif"))[0]
to_clip = [land_cover, local_inc, elipse_inc, ls_map, vh, vv, dem]
to_clip

In [ ]:
from typing import Union

def get_epsg(path: Union[str, Path]) -> str:
    """
    returns the EPSG of a geotiff
    """
    info = gdal.Info(str(path), format='json')
    return info['coordinateSystem']['wkt'].split('ID')[-1].split(',')[1][0:-2]

In [ ]:
f = gdal.Open(str(vh))
resolution = f.GetGeoTransform()[1]
src_epsg = get_epsg(land_cover)
dst_epsg = get_epsg(vh)
reproj_path = land_cover.parent/f"{land_cover.stem.split('EPSG')[0]}EPSG-{dst_epsg}.tif"
reproj_path

In [ ]:
gdal.Warp(str(reproj_path), str(land_cover),
          srcSRS=f'EPSG:{src_epsg}', dstSRS=f'EPSG:{dst_epsg}',
          xRes=resolution, yRes=resolution, targetAlignedPixels=True)

to_clip.remove(land_cover)
to_clip.append(reproj_path)
land_cover = reproj_path
print(to_clip)

In [ ]:
for pth in to_clip:
    clip = output_dir/f"{pth.stem}_clip.tif"
    if clip.is_file():
        clip.unlink()
    subset_command = f'gdalwarp -cutline {str(shp[0])} -crop_to_cutline {str(pth)} {str(clip)}'
    if pth == land_cover:
        land_cover = clip
    !$subset_command

## Mask out water bodies

- Permanent water bodies: 80
- Open sea: 200

In [ ]:
out = land_cover.parent/f"{land_cover.stem}_masked.tif"
shutil.copy(land_cover, out)

lc = gdal.Open(str(out), gdal.GA_Update)
lc_array = lc.ReadAsArray()

for val in [80, 200]:
    land_cover_masked = ma.masked_values(lc_array, val)
    lc_array = land_cover_masked.filled(fill_value=0)
    
lc.GetRasterBand(1).WriteArray(lc_array)
lc.FlushCache()
    
land_cover.unlink()